In [1]:
# 导包
import torch
import os
import argparse
import pandas as pd
import tqdm
import numpy as np
from torch.nn.functional import avg_pool2d

# os.environ['NUMEXPR_MAX_THREADS'] = '48'

# 导入各个指标
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')

# task select
from target_model.task_select import get_dataloader_and_model,get_dataloader_and_model, \
    get_dataloader,get_models,get_infotopo_para

# utils
from ppsplit.utils import concat_weights, create_dir, load_json, save_json

# defense:
from ppsplit.defense.obfuscation.scheduler import Scheduler

/home/dengruijun/miniconda3/envs/drj-pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# config = load_json('./config/nopeek.json')
# config = load_json('./config/shredder.json')
# config = load_json('./config/cloak.json')
config = load_json('./config/uniform_noise.json')

print(config)
args = config['general']
config['defense']['device']=args['device']

{'defense': {'method': 'uniform_noise', 'client': {'model_name': 'resnet18', 'split_layer': 6, 'pretrained': False, 'optimizer': 'adam', 'lr': 0.0003, 'distribution': 'gaussian', 'mean': 0, 'sigma': 1}, 'server': {'model_name': 'resnet18', 'split_layer': 6, 'logits': 2, 'pretrained': False, 'lr': 0.0003, 'optimizer': 'adam'}, 'learning_rate': 0.01, 'total_epochs': 1, 'training_batch_size': 128, 'dataset': 'fairface', 'protected_attribute': 'data', 'prediction_attribute': 'gender', 'img_size': 128, 'split': False, 'test_batch_size': 64, 'exp_id': '1', 'exp_keys': ['client.distribution', 'client.mean', 'client.sigma'], 'device': 'cuda:0'}, 'general': {'result_dir': '20241228-defense/', 'test_num': 'uniform_noise', 'device': 'cuda:0', 'dataset': 'CIFAR10', 'oneData_bs': 1, 'train_bs': 128, 'test_bs': 64, 'model': 'VGG5', 'split_layer': 2, 'ep': -1, 'no_dense': 0, 'noise_scale': 0}}


In [3]:
data_msg = get_dataloader(args)
model_msg = get_models(args)
infotopo_msg = get_infotopo_para(args)
msg = {**model_msg,**data_msg,**infotopo_msg}

# 数据集
one_data_loader,trainloader,testloader = data_msg['one_data_loader'],data_msg['trainloader'], data_msg['testloader']
data_interval = data_msg['data_interval']
data_type = msg['data_type']

# effectEntropy Infotopo参数
nb_of_values = msg['nb_of_values']

conv = msg['conv']
pool_size = msg['pool_size']
# conv = False
print("infotopo: nb_of_values: ",nb_of_values)

# 模型
client_net,decoder_net = model_msg['client_net'],model_msg['decoder_net']
server_net,unit_net = model_msg['server_net'], model_msg['unit_net']
decoder_route = model_msg['decoder_route']
image_deprocess = model_msg['image_deprocess']

# 路径
results_dir = model_msg['results_dir']
inverse_dir = results_dir + 'layer' + str(args['split_layer'])+'/'
# data_type = 1 if args['dataset'] == 'CIFAR10' else 0
split_layer = args['split_layer']

print('results_dir:', results_dir)
print('inverse_dir:', inverse_dir)
print('decoder_route:', decoder_route)

create_dir(inverse_dir)

# net使用
client_net = client_net.to(args['device'])
server_net = server_net.to(args['device'])
unit_net = unit_net.to(args['device'])


unit_net weights:  odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.5.weight', 'features.5.bias', 'features.5.running_mean', 'features.5.running_var', 'features.5.num_batches_tracked', 'features.8.weight', 'features.8.bias', 'features.9.weight', 'features.9.bias', 'features.9.running_mean', 'features.9.running_var', 'features.9.num_batches_tracked', 'denses.0.weight', 'denses.0.bias', 'denses.1.weight', 'denses.1.bias'])
client_net cweights:  odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.5.weight', 'features.5.bias', 'features.5.running_mean', 'features.5.running_var', 'features.5.num_batches_tracked'])
len unit_net weights:  25


In [4]:
# 防御
config['defense']["results_dir"] = results_dir
# config["1"] = results_dir
defense_scheduler = Scheduler(config)
if config['defense']['method']=='cloak':
    client_net = None
    server_net = unit_net

# 初始化和run存储模型
defense_scheduler.initialize(train_loader = trainloader, test_loader = testloader, client_model = client_net, server_model = server_net)
client_net,server_net = defense_scheduler.run_job()

# 拼接模型并保存
if client_net: # 如果有client_net，则拼接
    new_weights_unit = concat_weights(unit_net.state_dict(),client_net.state_dict(),server_net.state_dict())
else: # 如果没有client_net，则直接保存server_net
    new_weights_unit = server_net.state_dict()

unit_net.load_state_dict(new_weights_unit)
torch.save(unit_net.state_dict(), inverse_dir + 'unit_net_defensed.pth')

print("model saved in ",inverse_dir + 'unit_net_defensed.pth')

2025-01-01 04:42:33,978 - wandb.jupyter - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


server model device: cuda:0
client model device: cuda:0
algo_config {'model_name': 'resnet18', 'split_layer': 6, 'pretrained': False, 'optimizer': 'adam', 'lr': 0.0003, 'distribution': 'gaussian', 'mean': 0, 'sigma': 1, 'method': 'uniform_noise'}
Epoch:  0


100%|██████████| 391/391 [00:13<00:00, 29.01it/s]


tran loss: 1.4083401422061592


100%|██████████| 157/157 [00:02<00:00, 58.54it/s]


val loss: 1.2039988352234952
val acc: 0.5778264331210192
model saved in  ../../results/20241228-defense//VGG5/uniform_noise/layer2/unit_net_defensed.pth
